In [1]:
!pip install wandb
!pip install kaggle
!kaggle datasets download -d anon1729/aksharantar-sampled
!unzip aksharantar-sampled.zip

Dataset URL: https://www.kaggle.com/datasets/anon1729/aksharantar-sampled
License(s): unknown
 99%|█████████████████████████████████████▋| 14.0M/14.1M [00:01<00:00, 17.3MB/s]
100%|██████████████████████████████████████| 14.1M/14.1M [00:01<00:00, 10.9MB/s]
Archive:  aksharantar-sampled.zip
  inflating: aksharantar_sampled/asm/asm_test.csv  
  inflating: aksharantar_sampled/asm/asm_train.csv  
  inflating: aksharantar_sampled/asm/asm_valid.csv  
  inflating: aksharantar_sampled/ben/ben_test.csv  
  inflating: aksharantar_sampled/ben/ben_train.csv  
  inflating: aksharantar_sampled/ben/ben_valid.csv  
  inflating: aksharantar_sampled/brx/brx_test.csv  
  inflating: aksharantar_sampled/brx/brx_train.csv  
  inflating: aksharantar_sampled/brx/brx_valid.csv  
  inflating: aksharantar_sampled/guj/guj_test.csv  
  inflating: aksharantar_sampled/guj/guj_train.csv  
  inflating: aksharantar_sampled/guj/guj_valid.csv  
  inflating: aksharantar_sampled/hin/hin_test.csv  
  inflating: aksharantar_s

In [ ]:
import os
import wandb
import torch
import random
import argparse
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable

### Create an argument parser object
# parser=argparse.ArgumentParser()

### Define command-line arguments with descriptions, data types, default values, and choices where applicable
### WandB project name
# parser.add_argument('-wp',      '--wandb_project',      help='project name',                                                    type=str,       default='CS6910_A3_')

### WandB entity name
# parser.add_argument('-we',      '--wandb_entity',       help='entity name',                                                     type=str,       default='cs22m064'  )

### Number of Epochs
# parser.add_argument('-e',       '--epochs',             help='epochs',                          choices=[5,10],                 type=int,       default=5           )

### Batch Size
# parser.add_argument('-b',       '--batch_size',         help='batch sizes',                     choices=[32,64,128],            type=int,       default=32          )

### Optimizer Choice
# parser.add_argument('-o',       '--optimizer',          help='optimizer',                       choices=['adam','nadam'],       type=str,       default='adam'      )

### Learning Rate
# parser.add_argument('-lr',      '--learning_rate',      help='learning rates',                  choices=[1e-2,1e-3],            type=float,     default=1e-2        )

### Number Of layers in Encoder
# parser.add_argument('-nle',     '--num_layers_en',      help='number of layers in encoder',     choices=[1,2,3],                type=int,       default=2           )

### Number of layers in Decoder
# parser.add_argument('-nld',     '--num_layers_dec',     help='number of layers in decoder',     choices=[1,2,3],                type=int,       default=2           )
# parser.add_argument('-sz',      '--hidden_size',        help='hidden layer size',               choices=[16,32,64,256],         type=int,       default=256         )

### Add Input Language
# parser.add_argument('-il',      '--input_lang',         help='input language',                  choices=['hin','tel'],          type=str,       default='hin'       )

### Add target Language
# parser.add_argument('-tl',      '--target_lang',        help='target language',                 choices=['hin','tel'],          type=str,       default='hin'       )

### Select Cell Type :LSTM , GRU, RNN
# parser.add_argument('-ct',      '--cell_type',          help='cell type',                       choices=['LSTM','GRU','RNN'],   type=str,       default='LSTM'      )

### Select Drop out value
# parser.add_argument('-do',      '--drop_out',           help='drop out',                        choices=[0.0,0.2,0.3],          type=float,     default='0.2'       )

### Select Embedding size
# parser.add_argument('-es',      '--embedding_size',     help='embedding size',                  choices=[16,32,64,256],         type=int,       default=256         )

### Bidirectional true or false
# parser.add_argument('-bd',      '--bidirectional',      help='bidirectional',                   choices=[True,False],           type=bool,      default=False       )

### Attention True or False
# parser.add_argument('-at',      '--attention',          help='attention',                       choices=[True,False],           type=bool,      default=True        )

# args=parser.parse_args()


project_name_ap     = 'Assifnment_3_att'
entity_name_ap      = 'samruddhipatil2526'
# epochs_ap           = args.epochs
# batch_size_ap       = args.batch_size
# optimizer_ap        = args.optimizer
# learning_rate_ap    = args.learning_rate
# num_layers_en_ap    = args.num_layers_en
# num_layers_dec_ap   = args.num_layers_dec
# hidden_size_ap      = args.hidden_size
input_lang_ap       = 'eng'
target_lang_ap      = 'hin'
# cell_type_ap        = args.cell_type
# drop_out_ap         = args.drop_out
# embedding_size_ap   = args.embedding_size
bidirectional_ap    = False
attention_ap        = True

### Define the directory path where the data is located
dir = '/kaggle/working/aksharantar_sampled'


### Check if CUDA is available for GPU acceleration
use_cuda = torch.cuda.is_available()


# Define special tokens for the vocabulary
### Start-of-sequence token
SOS_token = 0
### End-of-sequence token
EOS_token = 1
### Unknown token
UNK_token = 3
### Padding token
PAD_token = 4

### # Define the sweep configuration using Bayesian optimization method
sweep_config ={
    'method':'bayes'
}

metric = {
    'name' : 'validation_accuracy',
    'goal' : 'maximize'
}
sweep_config['metric'] = metric

parameters_dict={
    'epochs':{
        'values' : [5,10]
    },
    'hidden_size':{
        'values' : [128,256,512]
    },
    'cell_type':{
        'values' : ['LSTM','RNN','GRU']
    },
    'learning_rate':{
        'values' : [1e-2,1e-3]
    },
    'num_layers_en':{
        'values' : [1,2,3]
    },
    'num_layers_dec':{
        'values' : [1,2,3]
    },
    'drop_out':{
        'values' : [0.0,0.2,0.3]
    },
    'embedding_size':{
        'values' : [64,128,256]
    },
    'batch_size':{
        'values' : [32,64,128]
    },
    'optimizer':{
        'values' : ['adam','nadam']
    },
    'bidirectional':{
        'values' : [True,False]
    }
}
sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project=project_name_ap)
print(sweep_id)

class Vocabulary:

    def __init__(self, name):
        self.char2count = {}
        self.char2index = {}
        self.n_chars = 4
        self.index2char = {0: '<', 1: '>',2 : '?', 3:'.'}
        self.name = name

    def addWord(self, word):
        for char in word:
            if char not in self.char2index:
                self.char2index[char] = self.n_chars
                self.index2char[self.n_chars] = char
                self.char2count[char] = 1
                self.n_chars += 1
            else:
                self.char2count[char] += 1
        

def prepareData(dir, lang1, lang2):

    data = pd.read_csv(dir,sep=",",names=['input', 'target'])
    max_input_length = max([len(txt) for txt in data['input'].to_list()])
    max_target_length = max([len(txt) for txt in data['target'].to_list()])

    input_lang = Vocabulary(lang1)
    output_lang = Vocabulary(lang2)

    pairs = []
    input_list,target_list = data['input'].to_list(),data['target'].to_list()
    for i in range(len(input_list)):
        pairs.append([input_list[i],target_list[i]])

    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])

    prepared_data = {
        'input_lang' : input_lang,
        'output_lang' : output_lang,
        'pairs' : pairs,
        'max_input_length' : max_input_length,
        'max_target_length' : max_target_length
    }

    return prepared_data

class EncoderRNN(nn.Module):
    """
        Initialize the EncoderRNN module.

        Args:
            input_size (int): Size of the input vocabulary.
            configuration (dict): Configuration dictionary containing model parameters.

        Configuration Parameters:
            - embedding_size (int): Size of the embedding layer.
            - hidden_size (int): Size of the hidden state.
            - num_layers_encoder (int): Number of layers in the encoder.
            - cell_type (str): Type of RNN cell ('RNN', 'GRU', or 'LSTM').
            - drop_out (float): Dropout rate.
            - bi_directional (bool): Whether the encoder is bidirectional.
        """
    def __init__(self, input_size, configuration):
        super(EncoderRNN, self).__init__()

        self.embedding_size = configuration['embedding_size']
        self.hidden_size = configuration['hidden_size']
        self.num_layers_encoder = configuration["num_layers_encoder"]
        self.cell_type = configuration["cell_type"]
        self.drop_out = configuration['drop_out']
        self.bi_directional = configuration['bi_directional']

        self.embedding = nn.Embedding(input_size, self.embedding_size)
        self.dropout = nn.Dropout(self.drop_out)
        self.cell_layer = None
        if self.cell_type == 'RNN':
            self.cell_layer = nn.RNN(self.embedding_size, self.hidden_size, num_layers = self.num_layers_encoder, dropout = self.drop_out, bidirectional = self.bi_directional)
        elif self.cell_type == 'GRU':
            self.cell_layer = nn.GRU(self.embedding_size, self.hidden_size, num_layers = self.num_layers_encoder, dropout = self.drop_out, bidirectional = self.bi_directional)
        elif self.cell_type == 'LSTM':
            self.cell_layer = nn.LSTM(self.embedding_size, self.hidden_size, num_layers = self.num_layers_encoder, dropout = self.drop_out, bidirectional = self.bi_directional)
 
    def forward(self, input, batch_size, hidden):
                """
        Forward pass of the EncoderRNN.

        Args:
            input (Tensor): Input tensor containing indices of input sequence.
            batch_size (int): Size of the input batch.
            hidden (Tensor): Initial hidden state.

        Returns:
            output (Tensor): Output tensor from the RNN layer.
            hidden (Tensor): Hidden state tensor.
        """
        embedded = self.dropout(self.embedding(input).view(1,batch_size, -1))
        output = embedded
        output, hidden = self.cell_layer(output, hidden)
        return output, hidden

    def initHidden(self ,batch_size, num_layers_enc):
        """
        Initialize the hidden state tensor.

        Args:
            batch_size (int): Size of the input batch.
            num_layers_enc (int): Number of layers in the encoder.

        Returns:
            Tensor: Initialized hidden state tensor.
        """
        res = None
        if self.bi_directional:
            res = torch.zeros(num_layers_enc* 2, batch_size, self.hidden_size)
        else:
            res = torch.zeros(num_layers_enc, batch_size, self.hidden_size)
        if use_cuda : 
            return res.cuda()
        else :
            return res

class DecoderRNN(nn.Module):
    def __init__(self, configuration,  output_size):
        """
        Initialize the DecoderRNN module.

        Args:
            configuration (dict): Configuration dictionary containing model parameters.
            output_size (int): Size of the output vocabulary.
        
        Configuration Parameters:
            - embedding_size (int): Size of the embedding layer.
            - hidden_size (int): Size of the hidden state.
            - num_layers_decoder (int): Number of layers in the decoder.
            - cell_type (str): Type of RNN cell ('RNN', 'GRU', or 'LSTM').
            - drop_out (float): Dropout rate.
            - bi_directional (bool): Whether the decoder is bidirectional.
        """
        super(DecoderRNN, self).__init__()

        self.embedding_size = configuration['embedding_size']
        self.hidden_size = configuration['hidden_size']
        self.num_layers_decoder = configuration["num_layers_decoder"]
        self.cell_type = configuration["cell_type"]
        self.drop_out = configuration["drop_out"]
        self.bi_directional = configuration["bi_directional"]
        self.dropout = nn.Dropout(self.drop_out)
        
        self.embedding = nn.Embedding(output_size, self.embedding_size)

        self.cell_layer = None
        if self.cell_type == 'RNN':
            self.cell_layer = nn.RNN(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out, bidirectional = self.bi_directional)
        elif self.cell_type == 'GRU':
            self.cell_layer =   nn.GRU(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out, bidirectional = self.bi_directional)
        elif self.cell_type == 'LSTM':
            self.cell_layer = nn.LSTM(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out, bidirectional = self.bi_directional)
        
        if self.bi_directional:
            self.out = nn.Linear(self.hidden_size * 2 ,output_size)
        else:
            self.out = nn.Linear(self.hidden_size, output_size)
        
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, batch_size, hidden):
        
        output = self.dropout(self.embedding(input).view(1,batch_size, -1))
        output = F.relu(output)
        output, hidden = self.cell_layer(output, hidden)
        
        output = self.softmax(self.out(output[0]))
        return output, hidden

class DecoderAttention(nn.Module) :

    def __init__(self, configs, output_size) :

        super(DecoderAttention, self).__init__()
        
        self.hidden_size = configs['hidden_size']
        self.embedding_size = configs['embedding_size']
        self.cell_type = configs['cell_type']
        self.num_layers_decoder = configs['num_layers_decoder']
        self.drop_out = configs['drop_out']
        self.max_length_word = configs['max_length_word']

        self.embedding = nn.Embedding(output_size, embedding_dim = self.embedding_size)
        self.attention_layer = nn.Linear(self.embedding_size + self.hidden_size, self.max_length_word + 1)
        self.attention_combine = nn.Linear(self.embedding_size + self.hidden_size, self.embedding_size)
        self.dropout = nn.Dropout(self.drop_out)

        self.cell_layer = None
        if self.cell_type == 'RNN':
            self.cell_layer = nn.RNN(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out)
        elif self.cell_type == 'GRU':
            self.cell_layer =   nn.GRU(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out)
        elif self.cell_type == 'LSTM':
            self.cell_layer = nn.LSTM(self.embedding_size, self.hidden_size, num_layers = self.num_layers_decoder, dropout = self.drop_out)
        self.out = nn.Linear(self.hidden_size, output_size)

    def forward(self, input, batch_size, hidden, encoder_outputs) :
        
        embedded = self.embedding(input).view(1, batch_size, -1)
        
        attention_weights = None
        if self.cell_type == 'LSTM' :
            attention_weights = F.softmax(self.attention_layer(torch.cat((embedded[0], hidden[0][0]), 1)), dim = 1)
        
        else :
            attention_weights = F.softmax(self.attention_layer(torch.cat((embedded[0], hidden[0]), 1)), dim = 1)

        attention_applied = torch.bmm(attention_weights.view(batch_size,1,self.max_length_word+1), encoder_outputs).view(1,batch_size,-1)
        output = torch.cat((embedded[0], attention_applied[0]), 1)
        output = self.attention_combine(output).unsqueeze(0)
        output = F.relu(output)
        output, hidden = self.cell_layer(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim = 1)
        
        return output, hidden, attention_weights

def indexesFromWord(lang, word):
    """
    Convert a word into a list of indices based on the character-to-index mapping of the language.

    Args:
        lang (Lang): Language object containing the character-to-index mapping.
        word (str): Input word to convert into indices.

    Returns:
        index_list (list): List of indices corresponding to the characters in the word.
    """
    index_list = []
    for char in word:
        if char in lang.char2index.keys():
            index_list.append(lang.char2index[char])
        else:
            index_list.append(UNK_token)
    return index_list

def variableFromSentence(lang, word, max_length):
    """
    Convert a word into a PyTorch tensor variable.

    Args:
        lang (Lang): Language object.
        word (str): Input word to convert.
        max_length (int): Maximum length of the sequence.

    Returns:
        result (Tensor): PyTorch tensor variable representing the input word.
    """
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    indexes.extend([PAD_token] * (max_length - len(indexes)))
    result = torch.LongTensor(indexes)
    if use_cuda:
        return result.cuda()
    else:
        return result

def variablesFromPairs(input_lang, output_lang, pairs, max_length):
    """
    Convert a list of word pairs into PyTorch tensor variables.

    Args:
        input_lang (Lang): Language object for the input language.
        output_lang (Lang): Language object for the output language.
        pairs (list): List of word pairs.
        max_length (int): Maximum length of the sequence.

    Returns:
        res (list): List of tuples containing input and target tensor variables.
    """
    res = []
    for pair in pairs:
        input_variable = variableFromSentence(input_lang, pair[0], max_length)
        target_variable = variableFromSentence(output_lang, pair[1], max_length)
        res.append((input_variable, target_variable))
    return res

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, configuration, max_length, teacher_forcing_ratio = 0.5):
    
    batch_size = configuration['batch_size']
    num_layers_enc = configuration['num_layers_encoder']
    encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

    input_tensor = Variable(input_tensor.transpose(0, 1))
    target_tensor = Variable(target_tensor.transpose(0, 1))

    if configuration["cell_type"] == "LSTM":
        encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
        encoder_hidden = (encoder_hidden, encoder_cell_state)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)

    decoder_input = Variable(torch.LongTensor([SOS_token]*batch_size))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden= decoder(decoder_input, batch_size, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, batch_size,decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            decoder_input = torch.cat(tuple(topi))

            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_tensor[di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length
  
def cal_val_loss(encoder, decoder, input_tensor, target_tensor, configuration, criterion , max_length):

    with torch.no_grad():

        batch_size = configuration['batch_size']
        num_layers_enc = configuration['num_layers_encoder']
        encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

        input_tensor = Variable(input_tensor.transpose(0, 1))
        target_tensor = Variable(target_tensor.transpose(0, 1))
            
        if configuration["cell_type"] == "LSTM":
            encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
            encoder_hidden = (encoder_hidden, encoder_cell_state)

        input_length = input_tensor.size()[0]
        target_length = target_tensor.size()[0]

        loss = 0
            
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)

        decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, batch_size, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            decoder_input = torch.cat(tuple(topi))

            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            loss += criterion(decoder_output, target_tensor[di])

    return loss.item() / target_length

def evaluate(encoder, decoder, loader, configuration, criterion , max_length):

    with torch.no_grad():

        batch_size = configuration['batch_size']
        total = 0
        correct = 0
        
        for batch_x, batch_y in loader:
            num_layers_enc = configuration['num_layers_encoder']
            encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

            input_variable = Variable(batch_x.transpose(0, 1))
            target_variable = Variable(batch_y.transpose(0, 1))
            
            if configuration["cell_type"] == "LSTM":
                encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
                encoder_hidden = (encoder_hidden, encoder_cell_state)

            input_length = input_variable.size()[0]
            target_length = target_variable.size()[0]

            output = torch.LongTensor(target_length, batch_size)

            encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
            encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
            
            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_variable[ei], batch_size, encoder_hidden)

            decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            decoder_hidden = encoder_hidden

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, batch_size, decoder_hidden)
                topv, topi = decoder_output.data.topk(1)
                decoder_input = torch.cat(tuple(topi))
                output[di] = torch.cat(tuple(topi))

            output = output.transpose(0,1)
            for di in range(output.size()[0]):
                ignore = [SOS_token, EOS_token, PAD_token]
                sent = [configuration['output_lang'].index2char[letter.item()] for letter in output[di] if letter not in ignore]
                y = [configuration['output_lang'].index2char[letter.item()] for letter in batch_y[di] if letter not in ignore]
                if sent == y:
                    correct += 1
                total += 1

    return (correct/total)*100

def train_with_attn(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, configuration, max_length, teacher_forcing_ratio = 0.5):
    
    batch_size = configuration['batch_size']
    num_layers_enc = configuration['num_layers_encoder']
    encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

    input_tensor = Variable(input_tensor.transpose(0, 1))
    target_tensor = Variable(target_tensor.transpose(0, 1))

    if configuration["cell_type"] == "LSTM":
        encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
        encoder_hidden = (encoder_hidden, encoder_cell_state)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]

    decoder_input = Variable(torch.LongTensor([SOS_token]*batch_size))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention= decoder(decoder_input, batch_size, decoder_hidden, encoder_outputs.reshape(batch_size,max_length, encoder.hidden_size))
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, batch_size,decoder_hidden, encoder_outputs.reshape(batch_size,max_length, encoder.hidden_size))
            topv, topi = decoder_output.data.topk(1)
            decoder_input = torch.cat(tuple(topi))

            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_tensor[di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def evaluate_with_attn(encoder, decoder, loader, configuration, criterion , max_length):

    with torch.no_grad():

        batch_size = configuration['batch_size']
        total = 0
        correct = 0
        
        for batch_x, batch_y in loader:

            num_layers_enc = configuration['num_layers_encoder']
            encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

            input_variable = Variable(batch_x.transpose(0, 1))
            target_variable = Variable(batch_y.transpose(0, 1))
            
            if configuration["cell_type"] == "LSTM":
                encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
                encoder_hidden = (encoder_hidden, encoder_cell_state)

            input_length = input_variable.size()[0]
            target_length = target_variable.size()[0]

            output = torch.LongTensor(target_length, batch_size)

            encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
            encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
            
            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_variable[ei], batch_size, encoder_hidden)

            decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            decoder_hidden = encoder_hidden

            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, batch_size, decoder_hidden,encoder_outputs.reshape(batch_size,max_length, encoder.hidden_size))
                topv, topi = decoder_output.data.topk(1)
                decoder_input = torch.cat(tuple(topi))
                output[di] = torch.cat(tuple(topi))

            output = output.transpose(0,1)
            for di in range(output.size()[0]):
                ignore = [SOS_token, EOS_token, PAD_token]
                sent = [configuration['output_lang'].index2char[letter.item()] for letter in output[di] if letter not in ignore]
                y = [configuration['output_lang'].index2char[letter.item()] for letter in batch_y[di] if letter not in ignore]
                if sent == y:
                    correct += 1
                total += 1

    return (correct/total)*100

def cal_val_loss_with_attn(encoder, decoder, input_tensor, target_tensor, configuration, criterion , max_length):

    with torch.no_grad():

        batch_size = configuration['batch_size']

        num_layers_enc = configuration['num_layers_encoder']
        encoder_hidden = encoder.initHidden(batch_size,num_layers_enc)

        input_tensor = Variable(input_tensor.transpose(0, 1))
        target_tensor = Variable(target_tensor.transpose(0, 1))
            
        if configuration["cell_type"] == "LSTM":
            encoder_cell_state = encoder.initHidden(batch_size,num_layers_enc)
            encoder_hidden = (encoder_hidden, encoder_cell_state)

        input_length = input_tensor.size()[0]
        target_length = target_tensor.size()[0]

        encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

        loss = 0
            
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)

        decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, batch_size, decoder_hidden, encoder_outputs.reshape(batch_size,max_length, encoder.hidden_size))
            topv, topi = decoder_output.data.topk(1)
            decoder_input = torch.cat(tuple(topi))

            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            loss += criterion(decoder_output, target_tensor[di])

    return loss.item() / target_length

def trainIters(encoder, decoder, train_loader, val_loader, test_loader, learning_rate, configuration, wandb_flag):

    max_length = configuration['max_length_word']

    train_loss_list, val_loss_list, train_acc_list, val_acc_list = [],[],[],[]

    encoder_optimizer, decoder_optimizer = None, None

    if configuration['optimizer']=='nadam':
        encoder_optimizer = optim.NAdam(encoder.parameters(),lr=learning_rate)
        decoder_optimizer = optim.NAdam(decoder.parameters(),lr=learning_rate)
    else:
        encoder_optimizer = optim.Adam(encoder.parameters(),lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(),lr=learning_rate)

    criterion = nn.NLLLoss()
    
    ep = configuration['epochs']

    for i in range(ep):
        
        if i % 5 == 0:
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Current Time = ", current_time)

        train_loss_total = 0
        val_loss_total = 0

        for batchx, batchy in train_loader:
            loss = None

            if configuration['attention'] == False:
                loss = train(batchx, batchy, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, configuration, max_length)
            else:
                loss = train_with_attn(batchx, batchy, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, configuration, max_length + 1)
            
            train_loss_total += loss
        
        train_loss_total = train_loss_total/len(train_loader)
        print('ep : ', i, ' | ', end='')
        print('train loss :', train_loss_total, ' | ', end='')

        for batchx, batchy in val_loader:
            loss = None

            if configuration['attention'] == False:
                loss = cal_val_loss(encoder, decoder, batchx, batchy, configuration, criterion , max_length)
            else:
                loss = cal_val_loss_with_attn(encoder, decoder, batchx, batchy, configuration, criterion , max_length+1)
            
            val_loss_total += loss

        val_loss_total = val_loss_total/len(val_loader)

        # train_acc = 0
        val_acc = 0
        
        # if configuration['attention'] == False:
        #     train_acc = evaluate(encoder, decoder, train_loader, configuration, criterion, max_length)
        # else:
        #     train_acc = evaluate_with_attn(encoder, decoder, train_loader, configuration, criterion, max_length+1)

        if configuration['attention'] == False:
            val_acc = evaluate(encoder, decoder, val_loader, configuration, criterion, max_length)
        else:
            val_acc = evaluate_with_attn(encoder, decoder, val_loader, configuration, criterion, max_length+1)
        
        # print("train accuracy : " ,train_acc, ' | ', end='')
        print('val loss :', val_loss_total, ' | ', end='')
        print("val accuracy : " ,val_acc)

        if wandb_flag == True:
            wandb.log({
                "train_loss"           : train_loss_total,
                "validation_loss"      : val_loss_total,
                # "train_accuracy"       : train_acc,
                "validation_accuracy"  : val_acc
                })

    # temp = configuration['batch_size']
    # configuration['batch_size'] = 1
    # if configuration['attention'] == False:
    #     print("test accuracy for the model : " ,evaluate(encoder, decoder, test_loader, configuration, criterion, max_length))
    # else:
    #     print("test accuracy for the model : " ,evaluate_with_attn(encoder, decoder, test_loader, configuration, criterion, max_length+1))
    # configuration['batch_size'] = temp


def main(config = None):

    with wandb.init(config = config, entity = entity_name_ap) as run:
        
        config = wandb.config
        run.name = 'hs_'+str(config.hidden_size)+'_bs_'+str(config.batch_size)+'_ct_'+config.cell_type+'_es_'+str(config.embedding_size)+'_do_'+str(config.drop_out)+'_nle_'+str(config.num_layers_en)+'_nld_'+str(config.num_layers_dec)+'_lr_'+str(config.learning_rate)+'_bd_'+str(config.bidirectional)

        configuration = {

                'hidden_size'         : config.hidden_size,
                'source_lang'         : input_lang_ap,
                'target_lang'         : target_lang_ap,
                'cell_type'           : config.cell_type,
                'num_layers_encoder'  : config.num_layers_en,
                'num_layers_decoder'  : config.num_layers_en,
                'drop_out'            : config.drop_out, 
                'embedding_size'      : config.embedding_size,
                'bi_directional'      : bidirectional_ap,
                'batch_size'          : config.batch_size,
                'attention'           : attention_ap,
                'epochs'              : config.epochs,
                'optimizer'           : config.optimizer

                # 'hidden_size'         : hidden_size_ap,
                # 'source_lang'         : input_lang_ap,
                # 'target_lang'         : target_lang_ap,
                # 'cell_type'           : cell_type_ap,
                # 'num_layers_encoder'  : num_layers_en_ap,
                # 'num_layers_decoder'  : num_layers_dec_ap,
                # 'drop_out'            : drop_out_ap, 
                # 'embedding_size'      : embedding_size_ap,
                # 'bi_directional'      : bidirectional_ap,
                # 'batch_size'          : batch_size_ap,
                # 'attention'           : attention_ap
            }
        
        
        train_path = os.path.join(dir, configuration['target_lang'], configuration['target_lang'] + '_train.csv')
        validation_path = os.path.join(dir, configuration['target_lang'], configuration['target_lang'] + '_valid.csv')
        test_path = os.path.join(dir, configuration['target_lang'], configuration['target_lang'] + '_test.csv')

        train_prepared_data= prepareData(train_path,configuration['source_lang'], configuration['target_lang'])

        input_lang = train_prepared_data['input_lang']
        output_lang = train_prepared_data['output_lang']
        pairs = train_prepared_data['pairs']
        max_input_length = train_prepared_data['max_input_length']
        max_target_length = train_prepared_data['max_target_length']
        
        val_prepared_data= prepareData(validation_path,configuration['source_lang'], configuration['target_lang'])

        val_pairs = val_prepared_data['pairs']
        max_input_length_val = val_prepared_data['max_input_length']
        max_target_length_val = val_prepared_data['max_target_length']

        test_prepared_data= prepareData(validation_path, configuration['source_lang'], configuration['target_lang'])

        test_pairs = test_prepared_data['pairs']
        max_input_length_test = test_prepared_data['max_input_length']
        max_target_length_test = test_prepared_data['max_target_length']

        max_list = [max_input_length, max_target_length, max_input_length_val, max_target_length_val, max_input_length_test, max_target_length_test]
        max_len_all = max(max_list)

        max_len = max(max_input_length, max_target_length) + 2

        configuration['input_lang'] = input_lang
        configuration['output_lang'] = output_lang
        configuration['max_length_word'] = max_len_all + 1

        print(configuration['max_length_word'])

        encoder1 = EncoderRNN(input_lang.n_chars, configuration)
        decoder1 = DecoderRNN(configuration, output_lang.n_chars)
        attndecoder1 = DecoderAttention(configuration, output_lang.n_chars)
        if use_cuda:
            encoder1=encoder1.cuda()
            decoder1=decoder1.cuda()
            attndecoder1 = attndecoder1.cuda()

        pairs = variablesFromPairs(configuration['input_lang'], configuration['output_lang'], pairs , configuration['max_length_word'])
        val_pairs = variablesFromPairs(configuration['input_lang'], configuration['output_lang'], val_pairs, configuration['max_length_word'])
        test_pairs = variablesFromPairs(configuration['input_lang'], configuration['output_lang'], test_pairs, configuration['max_length_word'])

        train_loader = torch.utils.data.DataLoader(pairs, batch_size=configuration['batch_size'], shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_pairs, batch_size=configuration['batch_size'], shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_pairs, batch_size=1, shuffle=True)

        if configuration['attention'] == False :
            trainIters(encoder1, decoder1, train_loader, val_loader, test_loader, config.learning_rate, configuration,True)
        else : 
            trainIters(encoder1, attndecoder1, train_loader, val_loader, test_loader, config.learning_rate, configuration,True)

# main()
wandb.agent(sweep_id, main, count = 10)

Create sweep with ID: bkiq3ufb
Sweep URL: https://wandb.ai/samruddhipatil2526/Assifnment_3_att/sweeps/bkiq3ufb
bkiq3ufb


wandb: Agent Starting Run: s5j7kqvs with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	drop_out: 0.2
wandb: 	embedding_size: 128
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_layers_dec: 3
wandb: 	num_layers_en: 2
wandb: 	optimizer: adam
wandb: Currently logged in as: samruddhipatil2526. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


25
Current Time =  08:10:36
ep :  0  | train loss : 1.1821789081573477  | val loss : 0.9744023311138154  | val accuracy :  0.0244140625
ep :  1  | train loss : 0.8786732283115377  | val loss : 0.6689877718687057  | val accuracy :  2.001953125
ep :  2  | train loss : 0.6334296529293062  | val loss : 0.538002740740776  | val accuracy :  10.302734375
ep :  3  | train loss : 0.5260157146692285  | val loss : 0.4738015508651734  | val accuracy :  14.5263671875
ep :  4  | train loss : 0.4716657031774518  | val loss : 0.4373766297101975  | val accuracy :  15.380859375
Current Time =  08:16:24
ep :  5  | train loss : 0.43158332815170264  | val loss : 0.4312347227334976  | val accuracy :  17.7490234375
ep :  6  | train loss : 0.4126716122627264  | val loss : 0.4131257092952728  | val accuracy :  19.1162109375
ep :  7  | train loss : 0.39634008834362044  | val loss : 0.4049714350700379  | val accuracy :  22.5830078125
ep :  8  | train loss : 0.3818129067659373  | val loss : 0.40082434535026545  |

train_loss,█▅▃▂▂▂▁▁▁▁
validation_accuracy,▁▂▄▅▅▆▇███
validation_loss,█▄▃▂▂▂▁▁▁▁
train_loss,0.36035
validation_accuracy,23.92578
validation_loss,0.38004


wandb: Agent Starting Run: 8eu8kqbc with config:
wandb: 	batch_size: 32
wandb: 	bidirectional: True
wandb: 	cell_type: RNN
wandb: 	drop_out: 0
wandb: 	embedding_size: 64
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_layers_dec: 3
wandb: 	num_layers_en: 3
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

wandb: Agent Starting Run: j3et5ja7 with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: GRU
wandb: 	drop_out: 0.2
wandb: 	embedding_size: 256
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.01
wandb: 	num_layers_dec: 3
wandb: 	num_layers_en: 2
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


25
Current Time =  09:45:54
ep :  0  | train loss : 0.877634793853761  | val loss : 0.6848592925071717  | val accuracy :  1.5380859375
ep :  1  | train loss : 0.686649202442169  | val loss : 0.6231478953361512  | val accuracy :  4.8583984375
ep :  2  | train loss : 0.658397208929062  | val loss : 0.6096479570865632  | val accuracy :  5.6396484375
ep :  3  | train loss : 0.6566326152324664  | val loss : 0.587642247080803  | val accuracy :  5.2734375
ep :  4  | train loss : 0.6318361799240111  | val loss : 0.5877847999334336  | val accuracy :  4.78515625
Current Time =  09:51:28
ep :  5  | train loss : 0.6222260604858397  | val loss : 0.5847256541252135  | val accuracy :  7.6416015625
ep :  6  | train loss : 0.6353045210838315  | val loss : 0.5899566870927813  | val accuracy :  4.58984375
ep :  7  | train loss : 0.6342987348556517  | val loss : 0.5974541026353838  | val accuracy :  6.2744140625
ep :  8  | train loss : 0.6306192460536958  | val loss : 0.6079938358068466  | val accuracy : 

train_loss,█▃▂▂▁▁▁▁▁▁
validation_accuracy,▁▅▆▅▅█▄▆▆▇
validation_loss,█▄▃▁▁▁▁▂▃▂
train_loss,0.6323
validation_accuracy,6.64062
validation_loss,0.59771


wandb: Agent Starting Run: twen9d0r with config:
wandb: 	batch_size: 32
wandb: 	bidirectional: False
wandb: 	cell_type: GRU
wandb: 	drop_out: 0
wandb: 	embedding_size: 128
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.01
wandb: 	num_layers_dec: 3
wandb: 	num_layers_en: 3
wandb: 	optimizer: nadam
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


25
Current Time =  09:57:40
ep :  0  | train loss : 0.890531234836578  | val loss : 0.6579412218928337  | val accuracy :  3.0029296875
ep :  1  | train loss : 0.6989523531913758  | val loss : 0.6505891025066373  | val accuracy :  2.9541015625
